In [1]:
import sys
sys.path.append('../')
from utilities import *

/home/palasso/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/palasso/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/palasso/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/palasso/.local/lib/python3.7/site-packages/tensorboard/compat

# Load data

In [2]:
corpus_dir = '../../data/corpus/'
model_dir = '../../data/ngrams/'

In [3]:
test = pd.read_csv(f'{corpus_dir}test.csv')

# Load Ngram Language Models

In [4]:
with open(f'{model_dir}counter.pickle', 'rb') as file:
    counter = pickle.load(file)

with open(f'{model_dir}vocabulary.pickle', 'rb') as file:
    vocabulary = pickle.load(file)

# Lidstone (add-a smoothing) trigram

In [5]:
trigramL = MLidstone(gamma=0.0051, order=3, vocabulary=vocabulary, counter=counter)

## Poisson Channel model

In [6]:
channel = NoisyChannelModel(trigramL, l=0.31, channel_method_poisson=True, channel_prob_param=0.01)
channel.show_progress = True

In [7]:
testL = channel.beam_search_sentences(test.err)

100%|██████████| 2500/2500 [4:35:31<00:00,  5.95s/it]  


In [8]:
testL.to_csv(f'{model_dir}testL.csv', index=False)

# Interpolated with Kneser-Ney smoothing trigram

In [9]:
trigramKNI = MKneserNeyInterpolated(order=3, discount=0.9284, vocabulary=vocabulary, counter=counter)

## Poisson Channel model

In [10]:
channel = NoisyChannelModel(trigramKNI, l=0.205, channel_method_poisson=True, channel_prob_param=0.01)
channel.show_progress = True

In [11]:
testKNI = channel.beam_search_sentences(test.err)

100%|██████████| 2500/2500 [8:01:19<00:00, 13.29s/it]   


In [12]:
testKNI.to_csv(f'{model_dir}testKNI.csv', index=False)